## Read .xyz point cloud to .csv

In [1]:
import pandas as pd
import numpy as np

In [2]:
felt1_accepted = "preprocessing/felt1_akseptert.xyz"
felt1_rejected = "preprocessing/felt1_forkastet.xyz"

felt2_accepted = "preprocessing/felt2_akseptert.xyz"
felt2_rejected = "preprocessing/felt2_forkastet.xyz"

In [3]:
def xyz_to_dataframe(accepted_xyz, rejected_xyz):
    # Hold all data read
    read_data = []
    dataframes = []

    # Rejected data
    with open(rejected_xyz, 'r') as file:
        lines = file.readlines()
        for idx, line in enumerate(lines):
            columns = line.strip().split(" ")
            columns.append(0)
            read_data.append(np.array(columns, dtype=float))

            if (idx != 0) and ((idx % 100000) == 0):
                dataframe = pd.DataFrame(read_data, columns=["x", "y", "z", "thu", "tvu", "accepted"])
                dataframes.append(dataframe)
                read_data = []
        dataframe = pd.DataFrame(read_data, columns=["x", "y", "z", "thu", "tvu", "accepted"])
        dataframes.append(dataframe)


    # Accepted data
    with open(accepted_xyz, 'r') as file:
        lines = file.readlines()
        for idx, line in enumerate(lines):
            columns = line.strip().split(" ")
            columns.append(1)
            read_data.append(np.array(columns, dtype=float))

            if (idx != 0) and ((idx % 500000) == 0):
                dataframe = pd.DataFrame(read_data, columns=["x", "y", "z", "thu", "tvu", "accepted"])
                dataframes.append(dataframe)
                read_data = []
        dataframe = pd.DataFrame(read_data, columns=["x", "y", "z", "thu", "tvu", "accepted"])
        dataframes.append(dataframe)

    # Combine all dataframes
    data = pd.concat(dataframes, ignore_index=True)
    data = data.sort_values(by=["x", "y", "z"], ascending=True)
    data = data[(data["thu"] >= 0) & (data["tvu"] >= 0)]
    data.x -= np.min(data.x)
    data.y -= np.min(data.y)
    data.z -= np.min(data.z)

    return data

In [4]:
felt1_data = xyz_to_dataframe(felt1_accepted, felt1_rejected)
felt2_data = xyz_to_dataframe(felt2_accepted, felt2_rejected)

In [5]:
felt1_data.to_csv("data/felt1_points.csv", index=False)
felt2_data.to_csv("data/felt2_points.csv", index=False)